In [39]:
import pandas as pd
import re
from datasets import Dataset

## Load dataset

In [19]:
data_path = "../dataset/business_proposal.csv"
bs_transcript_df = pd.read_csv(data_path)
bs_transcript_df.head()

,name,line
0,Receptionist,"Oh! I wasn’t eavesdropping, I swear!"
1,Receptionist,"You have a meeting… I mean, you *had* a meetin..."
2,Receptionist,I pressed the wrong button again. Sorry!
3,Receptionist,I always mess up when he walks in. He’s too ha...
4,Receptionist,I can’t believe I spilled coffee on the CEO’s ...


In [20]:
#Remove paranthesis
def remove_paranthesis(text):
    result = re.sub(r'\(.*?\)', '', text)

    return result

In [21]:
bs_transcript_df['line'] = bs_transcript_df['line'].apply(remove_paranthesis)

In [22]:
bs_transcript_df.head()

,name,line
0,Receptionist,"Oh! I wasn’t eavesdropping, I swear!"
1,Receptionist,"You have a meeting… I mean, you *had* a meetin..."
2,Receptionist,I pressed the wrong button again. Sorry!
3,Receptionist,I always mess up when he walks in. He’s too ha...
4,Receptionist,I can’t believe I spilled coffee on the CEO’s ...


In [23]:
bs_transcript_df['number_of_words'] = bs_transcript_df['line'].str.strip().str.split(" ")
bs_transcript_df['number_of_words'] = bs_transcript_df['number_of_words'].apply(lambda x: len(x))


In [24]:
bs_transcript_df.head()

,name,line,number_of_words
0,Receptionist,"Oh! I wasn’t eavesdropping, I swear!",6
1,Receptionist,"You have a meeting… I mean, you *had* a meetin...",13
2,Receptionist,I pressed the wrong button again. Sorry!,7
3,Receptionist,I always mess up when he walks in. He’s too ha...,11
4,Receptionist,I can’t believe I spilled coffee on the CEO’s ...,10


In [29]:
name = 'Kang Tae-moo'
bs_transcript_df["KangTaeMoo_respone_flag"] = 0
bs_transcript_df.loc[(bs_transcript_df['name'] == name) & (bs_transcript_df['number_of_words']>4), 'KangTaeMoo_respone_flag']=1 

In [30]:
bs_transcript_df

,name,line,number_of_words,KangTaeMoo_respone_flag
0,Receptionist,"Oh! I wasn’t eavesdropping, I swear!",6,0
1,Receptionist,"You have a meeting… I mean, you *had* a meetin...",13,0
2,Receptionist,I pressed the wrong button again. Sorry!,7,0
3,Receptionist,I always mess up when he walks in. He’s too ha...,11,0
4,Receptionist,I can’t believe I spilled coffee on the CEO’s ...,10,0
...,...,...,...,...
344,Shin Ha-Ri's Dad,He better have a stable income and stable emot...,9,0
345,Shin Ha-Ri's Dad,I raised you to stand tall—even in heels.,8,0
346,Shin Ha-Ri's Dad,"If he leaves you, I’ll chase him with this sli...",10,0
347,Shin Ha-Ri's Dad,"You shine, with or without him.",6,0


In [31]:
index_to_take = list(bs_transcript_df[(bs_transcript_df['KangTaeMoo_respone_flag']==1)&(bs_transcript_df.index>0)].index)

In [32]:
index_to_take[:5]

[35, 36, 37, 38, 39]

In [41]:
system_prompt = """ You are Kang Tae-moo from the movie "Business Proposal". Your respone should reflect his personality and speech partern \n"""

prompts = []
for ind in index_to_take:
    prompt = system_prompt

    prompt += bs_transcript_df.iloc[ind -1]['line']
    prompt += '\n'
    prompt += bs_transcript_df.iloc[ind]['line']
    prompts.append(prompt)

In [42]:
print(prompts[0])

 You are Kang Tae-moo from the movie "Business Proposal". Your respone should reflect his personality and speech partern 
Why is adulting allowed without training?
This isn't about business. It's about trust.


In [43]:
df = pd.DataFrame({"prompts": prompts})
df.head()

,prompts
0,"You are Kang Tae-moo from the movie ""Business..."
1,"You are Kang Tae-moo from the movie ""Business..."
2,"You are Kang Tae-moo from the movie ""Business..."
3,"You are Kang Tae-moo from the movie ""Business..."
4,"You are Kang Tae-moo from the movie ""Business..."


In [40]:
dataset = Dataset.from_pandas(df)